In [1]:
import sys
# print(sys.path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cooler
from mirnylib.genome import Genome
import cooltools.insulation as cool_insul
from bioframe import bedslice
%matplotlib qt5

import warnings
from cooltools.lib import peaks, numutils

In [2]:
! ls /net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DdeI-20161118.10000.cool

/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DdeI-20161118.10000.cool


In [2]:
c = cooler.Cooler('/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DdeI-20161118.10000.cool')

In [3]:
def get_chrom_arms(c, gen_name):
# Uses Genome class to get chromosomal regions
    genome = Genome('/net/levsha/share/lab/genomes/'+gen_name)

    name = []
    start = []
    end = []

    for ind, val in enumerate(genome.chrmArmLens):
        if ind%2==0:
            start.append(0)
            end.append(val)
            a = val
        else:
            start.append(a)
            end.append(val+a)
        name.append(c.chromnames[ind//2])
    
    del name[-1]
    end[-2] = end[-1]
    del end[-1]
    del start[-1]
    
    regions = list(zip(name, start, end))

    return regions
# regions = get_chrom_arms(c, 'hg19')
# region = regions[0]

## Testing Single Window Insulation Score from Cooltools

Testing full insulating code

In [4]:
result = cool_insul.find_insulating_boundaries(c, window_bp=[50000, 100000], min_dist_bad_bin=2, 
                                                         balance='weight', ignore_diags=None, 
                                                         chromosomes=None)

In [5]:
reg_res = bedslice(result.groupby('chrom'), region[0], region[1], region[2])

Testing insulating diamond code

In [ ]:
pix = c.matrix(balance=True, as_pixels=True).fetch(region,region)
bins = c.bins().fetch(region)

In [ ]:
ig_d = c._load_attrs(c.root.rstrip('/')+'/bins/weight')['ignore_diags']

In [ ]:
diam_res = cool_insul.insul_diamond(pix, bins, window=10, ignore_diags=ig_d, balanced=True, norm_by_median=True)
diam_res[diam_res==0] = np.nan
diam_res = np.log2(diam_res)

Compare both results

In [ ]:
plt.plot(diam_res)

In [ ]:
reg_res['log2_insulation_score_50000'].plot()

## Modifying find_insulating_boundaries so that it can iterate through multiple window sizes

In [ ]:
def find_insulating_boundaries(
    clr,
    window_bp=100000,
    min_dist_bad_bin=2, 
    balance='weight',
    ignore_diags=None,
    chromosomes=None,
):
    '''Calculate the diamond insulation scores and call insulating boundaries.

    Parameters
    ----------
    c : cooler.Cooler
        A cooler with balanced Hi-C data.
    window_bp : int or list
        The size of the sliding diamond window used to calculate the insulation
        score. If a list is provided, then a insulation score if done for each
        value of window_bp.
    min_dist_bad_bin : int
        The minimal allowed distance to a bad bin. Do not calculate insulation
        scores for bins having a bad bin closer than this distance.
    ignore_diags : int
        The number of diagonals to ignore. If None, equals the number of 
        diagonals ignored during IC balancing.

    Returns
    -------
    ins_table : pandas.DataFrame
        A table containing the insulation scores of the genomic bins and 
        the insulating boundary strengths.
    '''
    if chromosomes is None:
        chromosomes = clr.chromnames

    bin_size = clr.info['bin-size']
    ignore_diags = (ignore_diags 
        if ignore_diags is not None 
        else clr._load_attrs(clr.root.rstrip('/')+'/bins/weight')['ignore_diags'] )
    
    if isinstance(window_bp, int):
        window_bp = [window_bp]
    window_bp = np.array(window_bp)
    window_bins = window_bp // bin_size
    
    bad_win_sizes = window_bp % bin_size !=0 
    if np.any(bad_win_sizes):
        raise Exception(
            'The window sizes {} has to be a multiple of the bin size {}'.format(
                window_bp[bad_win_sizes], bin_size))
        
    ins_chrom_tables = []
    for chrom in chromosomes:
        chrom_bins = clr.bins().fetch(chrom)
        chrom_pixels = clr.matrix(as_pixels=True, balance=balance).fetch(chrom)
        
        is_bad_bin = np.isnan(chrom_bins['weight'].values)
        bad_bin_neighbor = np.zeros_like(is_bad_bin)
        for i in range(0, min_dist_bad_bin):
            if i == 0:
                bad_bin_neighbor = bad_bin_neighbor | is_bad_bin
            else:
                bad_bin_neighbor = bad_bin_neighbor | np.r_[[True]*i, is_bad_bin[:-i]]
                bad_bin_neighbor = bad_bin_neighbor | np.r_[is_bad_bin[i:], [True]*i]            

        ins_chrom = chrom_bins[['chrom', 'start', 'end']].copy()
        ins_chrom['bad_bin_masked'] = bad_bin_neighbor
        
        for j, win_bin in enumerate(window_bins):        
            with warnings.catch_warnings():                      
                warnings.simplefilter("ignore", RuntimeWarning)  
                ins_track = cool_insul.insul_diamond(chrom_pixels, chrom_bins, 
                    window=win_bin, ignore_diags=ignore_diags)
                ins_track[ins_track==0] = np.nan
                ins_track = np.log2(ins_track)

            ins_track[bad_bin_neighbor] = np.nan
            ins_track[~np.isfinite(ins_track)] = np.nan

            ins_chrom['log2_insulation_score_{}'.format(window_bp[j])] = ins_track

            poss, proms = peaks.find_peak_prominence(-ins_track)
            ins_prom_track = np.zeros_like(ins_track) * np.nan
            ins_prom_track[poss] = proms
            ins_chrom['boundary_strength_{}'.format(window_bp[j])] = ins_prom_track

        ins_chrom_tables.append(ins_chrom)

    ins_table = pd.concat(ins_chrom_tables)
    return ins_table

In [ ]:
%%time
result = cool_insul.find_insulating_boundaries(c, window_bp=[10000, 20000, 50000, 100000], min_dist_bad_bin=2, 
                                                         balance='weight', ignore_diags=None, 
                                                         chromosomes=None)

In [ ]:
reg_res = bedslice(result.groupby('chrom'), region[0], region[1], region[2])

Testing insulating diamond code

In [ ]:
pix = c.matrix(balance=True, as_pixels=True).fetch(region,region)
bins = c.bins().fetch(region)

In [ ]:
ig_d = c._load_attrs(c.root.rstrip('/')+'/bins/weight')['ignore_diags']

In [ ]:
diam_res = cool_insul.insul_diamond(pix, bins, window=2, ignore_diags=ig_d, balanced=True, norm_by_median=True)
diam_res[diam_res==0] = np.nan
diam_res = np.log2(diam_res)

Compare both results

In [ ]:
plt.plot(diam_res)

In [ ]:
reg_res['log2_insulation_score_20000'].plot()

## Comparing insulations scores of Hi-C and Micro-C

Hi-C for HFF cells

In [4]:
! ls /net/levsha/share/lab/dekkerU54/coolers/*ESC*.10000.cool

/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-DSG-20160722-DpnII.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-DSG-DdeI-20161014.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-DSG-DpnII-20170119.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-DSG-HindIII-20161206.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-20160812-DpnII.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DdeI-20161118.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DpnII-20170119.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-HindIII-20161206.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-FA-20160812-DpnII.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-FA81216DpnII-Hi.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-FA81216DpnII-Lo.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-FA81216DpnII-wash-Hi.10000.cool
/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-FA81216DpnII-wash-L

In [ ]:
clr = cooler.Cooler('/net/levsha/share/lab/dekkerU54/coolers/U54-ESC-EGS-DdeI-20161118.10000.cool')